In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import skimage.segmentation as seg
import skimage.filters as filt
import skimage.morphology as morph
import skimage.draw as draw
from scipy import ndimage
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import cumtrapz
import scipy.ndimage.filters as filters
import time
from scipy.spatial.distance import cdist
import itertools
import math
import pandas as pd
import os

d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def show(img_path):
  plt.imshow(img_path)
  plt.axis('off')
  plt.show()

  
def count_pos(arr):
  count = 0
  for num in arr:
    if num > 0:
        count += 1

  print("Number of positive elements in the array:", count)


In [3]:
def curvature(x, y, xc, yc, r):
    # Shift coordinates to make the center of the circle the origin
    x_shifted = x - xc
    y_shifted = y - yc
    # Calculate distance from each point to the center of the circle
    d = np.sqrt(x_shifted**2 + y_shifted**2)
    # Calculate the curvature only for points inside the circle
    inside_circle = d <= r
    dx_dt = np.gradient(x[inside_circle])
    dy_dt = np.gradient(y[inside_circle])
    d2x_dt2 = np.gradient(dx_dt)
    d2y_dt2 = np.gradient(dy_dt)
    denominator = dx_dt**2 + dy_dt**2
    curvature = np.zeros_like(denominator)
    nonzero_denominator = denominator != 0
    curvature[nonzero_denominator] = (dx_dt[nonzero_denominator] * d2y_dt2[nonzero_denominator] -
                                      d2x_dt2[nonzero_denominator] * dy_dt[nonzero_denominator]) / denominator[nonzero_denominator]**(3/2)
    # Pad the curvature array with zeros for the points outside the circle
    curvature_padded = np.zeros_like(d)
    curvature_padded[inside_circle] = curvature
    return curvature_padded


def integral_curvature(x, y, xc, yc, r):
    k = curvature(x, y, xc, yc, r)
    return cumtrapz(k, initial=0)


def get_integral_curvature(i, x, y, r, memo):
    # Check if the result has already been computed
    if (i, r) in memo:
        return memo[(i, r)]
    # Compute the integral curvature for a circle centered at (x[i], y[i]) with radius r
    int_curv = integral_curvature(x, y, x[i], y[i], r)[-1]
    # Memoize the result
    memo[(i, r)] = int_curv
    return int_curv


def get_area_measure(contour, i, radius, thresh, prev_center, prev_mask):
    # Compute current circle center and mask
    center = (contour[i][0][0], contour[i][0][1])
    circle_mask = np.zeros_like(gray)
    cv2.circle(circle_mask, center, radius, (255, 255, 255), -1)

    # Compute mask difference to find changed pixels
    diff_mask = cv2.absdiff(circle_mask, prev_mask)
    changed_pixels = np.where(diff_mask > 0)

    # Compute intersection area using only changed pixels
    intersection_mask = cv2.bitwise_and(circle_mask, thresh)
    changed_intersection_mask = intersection_mask[changed_pixels]
    intersection_area = np.sum(changed_intersection_mask) / 255

    # Update previous center and mask
    prev_center = center
    prev_mask = circle_mask

    # Compute area fraction
    circle_area = np.pi * radius ** 2
    intersection_fraction = intersection_area / circle_area

    # Return area fraction
    return intersection_fraction


def get_curvatures(contours, radius_arr, thresh):
    curvature_arr = []
    # Loop over each contour
    for contour in contours:
        curr_1 = []
        curr_2 = []

        # Precompute the coordinates of all points in the contour
        x, y = contour[:, 0, 0], contour[:, 0, 1]
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, [contour], 0, (255, 255, 255), -1)
        thresh = cv2.bitwise_and(gray, mask)
        # Compute the area measure for each point in the contour
        memo = {}
        for radius in radius_arr:
            prev_center = contour[0][0]
            prev_mask = np.zeros_like(gray)
            circle_perimeters = []
            curr_area_arr = []
            curr_int_arr = []
            for i in range(len(contour)):
                curr_area = get_area_measure(
                    contour, i, radius, thresh, prev_center, prev_mask)
                curr_int = get_integral_curvature(i, x, y, radius, memo)
                curr_int_arr.append(curr_int)
                curr_area_arr.append(curr_area)
            curr_1.append(curr_int_arr)
            curr_2.append(curr_area_arr)
        curvature_arr.append(curr_1)
        curvature_arr.append(curr_2)

    return curvature_arr


In [6]:
def resizeImage(img_path, new_width):
    # Load the image
    img = cv2.imread(img_path)
    # Get the image dimensions
    height, width = img.shape[:2]
    # Calculate the aspect ratio
    ratio = float(new_width) / width
    # Calculate the new height
    new_height = int(height * ratio)
    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height),
                             interpolation=cv2.INTER_AREA)
    return resized_img


def to_curv_image(currvature_arr, i, y):
    # Define image size and resolution
    img_width = 250
    img_height = 250

    # Create blank image
    curvature_img = np.zeros((img_height, img_width), np.uint8)

    # Define curvature array (example values)
    curvature_data = currvature_arr[i][y]

    # Map curvature values to pixel intensities
    max_curvature = np.max(np.abs(curvature_data))
    curvature_img = (np.abs(curvature_data) / max_curvature) * 255

    # Apply smoothing (optional)

    curvature_img = cv2.GaussianBlur(curvature_img, (5, 5), 0)
    return curvature_img


def to_hist(curvature_img):
    # Define histogram parameters
    hist_size = 21  # number of bins
    hist_range = (0, 256)  # range of values to use for the histogram
    curvature_img = np.uint8(curvature_img)
    # Calculate histogram
    histogram = cv2.calcHist([curvature_img], [0], None, [
                             hist_size], hist_range)
    return histogram


In [9]:
# specify path to the folder containing image files
radius_arr = np.arange(10, 131, 5)
path = "../../leafsnap/leafsnap-dataset/dataset/segmented/field"

counter = 0

# initialize empty lists for features and labels
hocs_area_list = []
hocs_int_list = []
labels = []
error_num = 0

# loop through all subfolders in the path
for foldername in os.listdir(path):

    folderpath = os.path.join(path, foldername)
    if not os.path.isdir(folderpath):
        continue

    # loop through all image files in the subfolder
    for filename in os.listdir(folderpath):

        filepath = os.path.join(folderpath, filename)
        if not os.path.isfile(filepath):
            continue

        try:

            start_time = time.time()
            # Load the image
            # resize the image
            img = resizeImage(filepath, 250)

            blurred_img = cv2.GaussianBlur(img, (5, 5), 0)
            # Convert the image to grayscale
            gray = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2GRAY)
            ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

            # Find the contours in the image
            contour, hierarchy = cv2.findContours(
                thresh,  cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            # Combine the contours
            contours = np.array(contour)
            combined_contour = np.concatenate(contours)

            # Compute centroids of contours
            centroids = []
            for c in contours:
                M = cv2.moments(c)
                if M['m00'] != 0:
                    cx = int(M['m10'] / M['m00'])
                    cy = int(M['m01'] / M['m00'])
                    centroids.append((cx, cy))

            # Compute distance matrix between centroids
            distances = np.zeros((len(centroids), len(centroids)))
            for i, j in itertools.combinations(range(len(centroids)), 2):
                dist = cv2.norm(np.array(centroids[i]), np.array(centroids[j]))
                distances[i, j] = dist
                distances[j, i] = dist

            # Find minimum spanning tree
            parent = [0]
            visited = [False] * len(centroids)
            visited[0] = True
            while len(parent) < len(centroids):
                min_dist = np.inf
                next_node = None
                for i in parent:
                    for j in range(len(centroids)):
                        if not visited[j] and distances[i, j] < min_dist:
                            min_dist = distances[i, j]
                            next_node = j
                parent.append(next_node)
                visited[next_node] = True

            # Reorder contours based on minimum spanning tree
            new_contours = [contours[i] for i in parent]

            start_time = time.time()

            currvature_arr = get_curvatures(new_contours, radius_arr, thresh)
            int_hist = []
            area_hist = []

            for y in range(len(currvature_arr[0])):
                curv_image_hist = to_curv_image(currvature_arr, 0, y)
                curv_image_area = to_curv_image(currvature_arr, 1, y)
                histogram_hist_int = to_hist(curv_image_hist)
                histogram_hist_area = to_hist(curv_image_area)

                int_hist.append(histogram_hist_int)
                area_hist.append(histogram_hist_area)

            flat_hist_int = np.array(int_hist).flatten()
            flat_hist_area = np.array(area_hist).flatten()

            hocs_area_list.append(flat_hist_area)
            hocs_int_list.append(flat_hist_int)
            labels.append(foldername)

            end_time = time.time()
            total_time = end_time - start_time

        except:
            error_num += 1
            print(error_num)
            pass

    print(f"Finished plant: {foldername}")

labels = np.array(labels)

# save features and labels to a pandas dataframe and export to CSV file
data = pd.DataFrame({"area_hist_values": hocs_area_list,
                    "int_hist_values": hocs_int_list})
data["plant"] = labels
data.to_csv("FINAL_HOCS_1.csv", index=False)


[  3.   6.  13.  23.  23.  24.  12.   7.   5.   8.   0.   1.   1.   0.
   0.   0.   1.   1.   0.   0.   0.   4.  13.  11.  23.  20.  14.   4.
   7.   7.   8.   8.   2.   0.   1.   1.   2.   1.   2.   0.   0.   0.
  17.  38.  32.  13.  10.   5.   8.   0.   0.   1.   1.   0.   1.   0.
   0.   2.   0.   0.   0.   0.   0.  33.  45.  30.   4.   5.   2.   1.
   1.   0.   3.   0.   2.   0.   0.   2.   0.   0.   0.   0.   0.   0.
  19.  41.  30.  19.   5.   6.   4.   0.   0.   0.   1.   0.   0.   1.
   2.   0.   0.   0.   0.   0.   0.  27.  49.  16.   9.  11.   3.   2.
   2.   1.   2.   0.   2.   0.   2.   2.   0.   0.   0.   0.   0.   0.
  17.  34.  19.  19.   5.  12.   2.   2.   4.   3.   1.   2.   3.   0.
   1.   0.   1.   0.   1.   2.   0.  28.  42.  19.   6.  11.   5.   3.
   2.   1.   2.   4.   1.   0.   2.   0.   0.   1.   1.   0.   0.   0.
  23.  33.  22.   8.   1.   5.  13.   3.   3.   3.   0.   1.   1.   2.
   3.   1.   1.   0.   2.   0.   3.  16.  24.  26.   7.   5.   5.   7.
   6. 